In [1]:

from __future__ import print_function

import sys
import socket

hostname = socket.gethostname()

if hostname == 'toologic-ubuntu2':
    sys.path.extend(['/home/jogi/git/repository/dcnn_mri_seg'])
elif hostname == 'qiaubuntu':
    sys.path.extend(['/home/jorg/repository/dcnn_mri_seg'])
else:
    raise ValueError("Unknown hostname {}".format(hostname))

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import HVSMR2016CardiacMRI
from utils.config import config, BaseConfig
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler, TestHandler

from models.dilated_cnn import BaseDilated2DCNN

from jelmer_util.helper import loadImageDir, generateBatch2D

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
dataset = HVSMR2016CardiacMRI(data_dir=config.data_dir,
                                  search_mask=config.dflt_image_name + ".nii",
                                  norm_scale=config.norm_method, load_type="numpy",
                                  val_fold=1)
dataset.get_test_image()

>>>>>>>>>>>> Info - Looking for files with search_mask /home/jorg/repository/dcnn_mri_seg/data/HVSMR2016/train/aug_*.npz
Info - cannot find any numpy npz files. Looking for raw files...
> > > Loading image+label from /home/jorg/repository/dcnn_mri_seg/data/HVSMR2016/train/0_image.nii
/home/jorg/repository/dcnn_mri_seg/data/HVSMR2016/train/0_image.nii / /home/jorg/repository/dcnn_mri_seg/data/HVSMR2016/train/0_label.nii
> > > Loading image+label from /home/jorg/repository/dcnn_mri_seg/data/HVSMR2016/test/9_image.nii


In [3]:
exper = ExperimentHandler.load_experiment("20180201_08_58_03_dcnnv1_5000E_lr1e03/stats/exper_stats@5000.dll")
new_config = BaseConfig()
new_config.copy_from_object(exper.config)
new_config.root_dir = os.environ.get("REPO_PATH")
exper.set_new_config(new_config)
exper_hdl = ExperimentHandler(exper)
model = exper_hdl.load_checkpoint(root_dir="/home/jorg/repository/dcnn_mri_seg/", checkpoint=5000)

Load from /home/jorg/repository/dcnn_mri_seg/logs/20180201_08_58_03_dcnnv1_5000E_lr1e03/stats/exper_stats@5000.dll


INFO - INFO - loaded existing model from checkpoint /home/jorg/repository/dcnn_mri_seg/logs/20180201_08_58_03_dcnnv1_5000E_lr1e03/checkpoints/BaseDilated2DCNNcheckpoint05000.pth.tar


In [19]:
exper_hdl.test_full_image(model, dataset.test_images[0], batch_size=5, spacing=dataset.test_spacings[0],
                          views=None, labels=None, gen_overlays=True)

INFO - Be patient - Generating overlays for axial view
('Image shape in/out ', (193, 272, 209), (209, 3, 193, 272), 'axial')
INFO - Generating overlays took 7.420 seconds 
INFO - Be patient - Generating overlays for coronal view
('Image shape in/out ', (193, 272, 209), (272, 3, 193, 209), 'coronal')
INFO - Generating overlays took 7.562 seconds 
INFO - Be patient - Generating overlays for saggital view
('Image shape in/out ', (193, 272, 209), (193, 3, 272, 209), 'saggital')
INFO - Generating overlays took 7.015 seconds 
[0 1 2]
('In method get_pred_class_labels, shape of input ', (3, 193, 272, 209))
Successfully saved image to /home/jorg/repository/dcnn_mri_seg/logs/20180201_08_58_03_dcnnv1_5000E_lr1e03/figures/test_myocardium.nii
Successfully saved image to /home/jorg/repository/dcnn_mri_seg/logs/20180201_08_58_03_dcnnv1_5000E_lr1e03/figures/test_bloodpool.nii
('Full image out shape ', (3, 209, 272, 193))
Successfully saved image to /home/jorg/repository/dcnn_mri_seg/logs/20180201_08_

In [14]:
del test_handler